### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [16]:
%%capture
# transformers 라이브러리 설치
!pip install transformers

In [17]:
# 필요한 라이브러리 임포트
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [18]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# 데이터 CSV 파일 로드
df = pd.read_csv('/content/bbc-text.csv') # bbc-text.csv 파일 경로

In [20]:
# 데이터프레임 첫 5행 출력
display(df.head())

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [21]:
# 데이터프레임 총 행 수 출력
print(len(df))

2225


In [22]:
# 카테고리별 기사 수 집계 및 출력
display(df.groupby('category').count())

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [23]:
# BERT 토크나이저 로드 및 카테고리 레이블 매핑
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

## Dataset

In [24]:
# PyTorch 커스텀 데이터셋 클래스 정의
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        # 레이블 정수 변환
        self.labels = [labels[label] for label in df['category']]
        # 텍스트 토큰화 및 패딩
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    # 데이터셋 길이 반환
    def __len__(self):
        return len(self.labels)

    # 배치 레이블 반환
    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    # 배치 텍스트 반환
    def get_batch_texts(self, idx):
        return self.texts[idx]

    # 인덱스에 해당하는 데이터 반환
    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [25]:
# BERT 분류 모델 클래스 정의
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        # 사전 학습된 BERT 모델 로드
        self.bert = BertModel.from_pretrained('bert-base-cased')
        # Dropout 레이어
        self.dropout = nn.Dropout(dropout)
        # 선형 분류 레이어
        self.linear = nn.Linear(768, 5)
        # ReLU 활성화 함수
        self.relu = nn.ReLU()

    # 순전파 메서드
    def forward(self, input_id, mask):
        # BERT 모델 순전파
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # Dropout 적용
        dropout_output = self.dropout(pooled_output)
        # 선형 레이어 적용
        linear_output = self.linear(dropout_output)
        # ReLU 적용
        final_layer = self.relu(linear_output)
        # 최종 출력 반환
        return final_layer

In [26]:
# 모델 학습 함수 정의
def train(model, train_data, val_data, learning_rate, epochs):
    # Dataset 인스턴스 생성
    train, val = Dataset(train_data), Dataset(val_data)

    # 데이터 로더 생성
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # 장치 설정 (GPU 또는 CPU)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 손실 함수 및 옵티마이저 정의
    criterion = nn.CrossEntropyLoss() # CrossEntropy loss
    optimizer = Adam(model.parameters(), lr= learning_rate) # Adam 최적화

    # 모델 및 손실 함수를 장치로 이동
    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    # 학습 루프
    for epoch_num in range(epochs):
            total_acc_train = 0
            total_loss_train = 0

            # 학습 데이터 반복
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                # 모델 순전파 및 손실 계산
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                # 정확도 계산
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # 역전파 및 가중치 업데이트
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            # 검증 루프
            with torch.no_grad():
                for val_input, val_label in val_dataloader:
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    # 모델 순전파 및 손실 계산
                    output = model(input_id, mask)
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    # 정확도 계산
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            # 에포크 결과 출력
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [27]:
# 모델 평가 함수 정의
def evaluate(model, test_data):
    # Dataset 인스턴스 생성
    test = Dataset(test_data)
    # 데이터 로더 생성
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    # 장치 설정 (GPU 또는 CPU)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 모델을 장치로 이동
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    # 평가 루프
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              # 모델 순전파
              output = model(input_id, mask)

              # 정확도 계산
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    # 최종 테스트 정확도 출력
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [28]:
# 데이터셋 분할 (학습, 검증, 테스트)
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

# 분할된 데이터셋 크기 출력
print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [29]:
# 학습 에포크 및 학습률 설정
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

# 모델 학습 실행
train(model, df_train, df_val, LR, EPOCHS)

  0%|          | 0/890 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
 57%|█████▋    | 509/890 [01:53<01:24,  4.50it/s]


KeyboardInterrupt: 

In [15]:
# 모델 평가 실행
evaluate(model, df_test)

Test Accuracy:  0.902


optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗